In [2]:
!pip install langchain

     ---------------------------------------- 1.0/1.0 MB 16.1 MB/s eta 0:00:00
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
     -------------------------------------- 161.8/161.8 kB 9.5 MB/s eta 0:00:00
     ------------------------------------- 360.3/360.3 kB 21.9 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 44.7 MB/s eta 0:00:00
     ------------------------------------- 437.7/437.7 kB 26.7 MB/s eta 0:00:00
     ------------------------------------- 495.5/495.5 kB 30.3 MB/s eta 0:00:00
     ---------------------------------------- 134.6/134.6 kB ? eta 0:00:00
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
     -------------------------------------- 295.0/295.0 kB 9.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install langchain_openai

     ---------------------------------------- 62.8/62.8 kB ? eta 0:00:00
     ------------------------------------- 894.0/894.0 kB 19.2 MB/s eta 0:00:00
     ------------------------------------- 274.0/274.0 kB 16.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%pip install -U langchain-community

     ---------------------------------------- 2.5/2.5 MB 32.3 MB/s eta 0:00:00
     ------------------------------------- 442.9/442.9 kB 27.0 MB/s eta 0:00:00
     ---------------------------------------- 44.4/44.4 kB ? eta 0:00:00
     ---------------------------------------- 92.9/92.9 kB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 120.8/120.8 kB ? eta 0:00:00
     ---------------------------------------- 45.2/45.2 kB ? eta 0:00:00
     ---------------------------------------- 50.9/50.9 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
%pip install pypdf

     -------------------------------------- 303.4/303.4 kB 9.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader, JSONLoader, CSVLoader,WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

1. 문서읽기
2. 문서 나누기(chunk size)
3. embedding
4. vector db 저장하기
5. 유사도검색을 통해 문서추출
6. 추출된문서(context )
Context: 유사도검색을 통해 검색된 문서들의 합본
Question:내질문
Answer:

In [13]:
pdfloader = PyPDFLoader( 'venv/data/med_data_all.pdf')
document = pdfloader.load()
print( document)

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-05-14T15:44:39+09:00', 'author': 'Nitrosoft', 'moddate': '2025-05-14T15:44:39+09:00', 'source': 'venv/data/med_data_all.pdf', 'total_pages': 126, 'page': 0, 'page_label': '1'}, page_content="A1C \nA1C is a blood test for type 2 diabetes and prediabetes. It measures your average blood \nglucose, or blood sugar, level over the past 3 months. Doctors may use the A1C alone or \nin combination with other diabetes tests to make a diagnosis. They also use the A1C to \nsee how well you are managing your diabetes. This test is different from the blood sugar \nchecks that people with diabetes do every day. Your A1C test result is given in \npercentages. The higher the percentage, the higher your blood sugar levels have been. A \nnormal A1C level is below 5.7%Prediabetes is between 5.7 to 6.4%. Having prediabetes is \na risk factor for getting type 2 diabetes. People with prediabetes m

In [14]:
len( document)

126

In [15]:
document

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-05-14T15:44:39+09:00', 'author': 'Nitrosoft', 'moddate': '2025-05-14T15:44:39+09:00', 'source': 'venv/data/med_data_all.pdf', 'total_pages': 126, 'page': 0, 'page_label': '1'}, page_content="A1C \nA1C is a blood test for type 2 diabetes and prediabetes. It measures your average blood \nglucose, or blood sugar, level over the past 3 months. Doctors may use the A1C alone or \nin combination with other diabetes tests to make a diagnosis. They also use the A1C to \nsee how well you are managing your diabetes. This test is different from the blood sugar \nchecks that people with diabetes do every day. Your A1C test result is given in \npercentages. The higher the percentage, the higher your blood sugar levels have been. A \nnormal A1C level is below 5.7%Prediabetes is between 5.7 to 6.4%. Having prediabetes is \na risk factor for getting type 2 diabetes. People with prediabetes m

In [16]:
document[0].metadata['page']

0

In [19]:
for doc in document:
    print( doc.page_content)
    print('------------')

A1C 
A1C is a blood test for type 2 diabetes and prediabetes. It measures your average blood 
glucose, or blood sugar, level over the past 3 months. Doctors may use the A1C alone or 
in combination with other diabetes tests to make a diagnosis. They also use the A1C to 
see how well you are managing your diabetes. This test is different from the blood sugar 
checks that people with diabetes do every day. Your A1C test result is given in 
percentages. The higher the percentage, the higher your blood sugar levels have been. A 
normal A1C level is below 5.7%Prediabetes is between 5.7 to 6.4%. Having prediabetes is 
a risk factor for getting type 2 diabetes. People with prediabetes may need retests every 
year. Type 2 diabetes is above 6.5%If you have diabetes, you should have the A1C test at 
least twice a year. The A1C goal for many people with diabetes is below 7. It may be 
different for you. Ask what your goal should be. If your A1C result is too high, you may 
need to change your dia

In [20]:
txt_split = RecursiveCharacterTextSplitter( chunk_size=500, chunk_overlap=20 )
docs = txt_split.split_documents( document )

In [21]:
docs

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-05-14T15:44:39+09:00', 'author': 'Nitrosoft', 'moddate': '2025-05-14T15:44:39+09:00', 'source': 'venv/data/med_data_all.pdf', 'total_pages': 126, 'page': 0, 'page_label': '1'}, page_content='A1C \nA1C is a blood test for type 2 diabetes and prediabetes. It measures your average blood \nglucose, or blood sugar, level over the past 3 months. Doctors may use the A1C alone or \nin combination with other diabetes tests to make a diagnosis. They also use the A1C to \nsee how well you are managing your diabetes. This test is different from the blood sugar \nchecks that people with diabetes do every day. Your A1C test result is given in'),
 Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-05-14T15:44:39+09:00', 'author': 'Nitrosoft', 'moddate': '2025-05-14T15:44:39+09:00', 'source': 'venv/data/med_data_all.pdf', 'total_p

In [22]:
len( docs)

789

In [23]:
embedding = OpenAIEmbeddings( model='text-embedding-ada-002')
vdb = FAISS.from_documents( docs, embedding )

In [24]:
vdb.save_local('myfaiss')
# FAISS.load_local('myfaiss')

In [26]:
gpt = ChatOpenAI( model='gpt-3.5-turbo', temperature=0)
retriever=  vdb.as_retriever()# 4개의 유사도가 기본값
qa = RetrievalQA.from_chain_type( llm=gpt, chain_type='stuff',
                            retriever=retriever ) 
query = '고혈압에 대해 알려줘'
result = qa.invoke( {'query':query})

print( result )

{'query': '고혈압에 대해 알려줘', 'result': '고혈압은 혈액이 동맥 벽에 너무 많은 압력을 가할 때 발생하는 질환으로, 미국 성인의 거의 절반이 고혈압을 가지고 있습니다. 대부분 증상이 없는데도 심장마비, 심근경색, 뇌졸중, 신부전과 같은 심각한 문제를 일으킬 수 있습니다. 건강한 생활습관 변화는 고혈압을 낮출 수 있는데, 체중 감량, 신체 활동, 스트레스 관리, 식단 속 나트륨 감소, 알코올과 담배, 불법 약물 피하기, 충분한 수면 등이 도움이 됩니다. 생활습관 변화만으로 고혈압을 낮출 수 없는 경우 의료진이 혈압약을 처방할 수 있습니다.'}


문서:
{context}

질문:
{question}

답변을 작성하세요.

OpenAI의 text-embedding-ada-002 모델 특성
-----------------------------------------------------
text-embedding-ada-002는 단순한 영어 전용이 아니라,
한국어, 영어, 일본어, 중국어 등 다양한 언어를
"공통된 의미 공간(shared semantic space)" 안에서 임베딩한다


"The Adventures of Tom Sawyer"라는 영어 텍스트를 임베딩하면,
그것이 가진 의미(semantics) 가 벡터로 표현됨.
그리고 한글 질문 "줄거리 요약해줘"도 벡터로 임베딩할 때
그 의미가 비슷한 벡터로 변환됨.

결국
"의미가 비슷하면 언어가 달라도 벡터가 가까워진다" 는 것.
